In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style

from path import Path
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, classification_report, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.tree import DecisionTreeRegressor

In [2]:
# path to dataset
file_path = Path("/Users/Emily/documents/class_data/final_group_airbnb/Resources/cleaned_airbnb_dataset4.csv")
airbnb_df = pd.read_csv(file_path, index_col=0)
airbnb_df.head()

,host_id,host_is_superhost,property_type,room_type,neighborhood,zipcode,accommodates,bedrooms,bathrooms,minimum_nights,...,latitude,longitude,reviews,review_scores_rating,price,cleaning_fee,total,Median,Mean,Pop
room_id,,,,,,,,,,,,,,,,,,,,,
958,1169,t,Apartment,Entire home/apt,Western Addition,94117,3,1.0,1.0,1,...,37.76931,-122.43386,217,97.0,170.0,100.0,270.0,84802.0,112291.0,39169.0
3850,4921,t,House,Private room,Inner Sunset,94131,2,1.0,1.0,1,...,37.75402,-122.45805,160,94.0,99.0,10.0,109.0,97834.0,126303.0,26881.0
5858,8904,f,Apartment,Entire home/apt,Bernal Heights,94110,5,2.0,1.0,30,...,37.74511,-122.42102,111,98.0,235.0,100.0,335.0,75106.0,98470.0,69333.0
7918,21994,t,Apartment,Private room,Haight Ashbury,94117,2,1.0,4.0,32,...,37.76669,-122.45250,18,86.0,65.0,50.0,115.0,84802.0,112291.0,39169.0
8142,21994,t,Apartment,Private room,Haight Ashbury,94117,2,1.0,4.0,32,...,37.76487,-122.45183,8,93.0,65.0,50.0,115.0,84802.0,112291.0,39169.0


In [3]:
# dropping categorical data
num_airbnb_df = airbnb_df.drop(['maximum_nights','reviews','cleaning_fee','total','Median','Mean','Pop','host_id','host_is_superhost','property_type','room_type',
                                'neighborhood','latitude','longitude','zipcode'], axis=1)

In [4]:
num_airbnb_df.head()

,accommodates,bedrooms,bathrooms,minimum_nights,review_scores_rating,price
room_id,,,,,,
958,3,1.0,1.0,1,97.0,170.0
3850,2,1.0,1.0,1,94.0,99.0
5858,5,2.0,1.0,30,98.0,235.0
7918,2,1.0,4.0,32,86.0,65.0
8142,2,1.0,4.0,32,93.0,65.0


In [5]:
#le = LabelEncoder()
#num_airbnb_df['accommodates'] = le.fit_transform(num_airbnb_df['accommodates'])
#num_airbnb_df['bedrooms'] = le.fit_transform(num_airbnb_df['bedrooms'])
#num_airbnb_df['bathrooms'] = le.fit_transform(num_airbnb_df['bathrooms'])
#num_airbnb_df['minimum_nights'] = le.fit_transform(num_airbnb_df['minimum_nights'])
#num_airbnb_df['review_scores_rating'] = le.fit_transform(num_airbnb_df['review_scores_rating'])
#num_airbnb_df['price'] = le.fit_transform(num_airbnb_df['price'])


Selected Features

In [6]:
feat2 = ['accommodates','bedrooms','bathrooms','minimum_nights','review_scores_rating']
X = num_airbnb_df[feat2]
Y = num_airbnb_df.price
X.head()

,accommodates,bedrooms,bathrooms,minimum_nights,review_scores_rating
room_id,,,,,
958,3,1.0,1.0,1,97.0
3850,2,1.0,1.0,1,94.0
5858,5,2.0,1.0,30,98.0
7918,2,1.0,4.0,32,86.0
8142,2,1.0,4.0,32,93.0


In [7]:
X_train, X_pred, Y_train, Y_pred = train_test_split(X, Y, random_state = 42)

In [8]:
# standard scaler
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

#print(scaler.fit(data))

In [9]:
#X_train_scaled = X_scaler.transform(X_train)
#X_pred_scaled = X_scaler.transform(X_pred)

In [10]:
print("Training set: X train:{} Y train:{}".format(X_train.shape, Y_train.shape))
print("Prediction set: X pred:{} Y pred:{}".format(X_pred.shape, Y_pred.shape))
print("Full dataset: X:{} Y:{}".format(X.shape, Y.shape))

Training set: X train:(5861, 5) Y train:(5861,)
Prediction set: X pred:(1954, 5) Y pred:(1954,)
Full dataset: X:(7815, 5) Y:(7815,)


In [12]:
num_airbnb_df = DecisionTreeRegressor(random_state = 42)
num_airbnb_df.fit(X_train, Y_train)

DecisionTreeRegressor(random_state=42)

In [13]:
print(X_pred)

          accommodates  bedrooms  bathrooms  minimum_nights  \
room_id                                                       
13284332             4       0.0        1.0               1   
38699317             3       1.0        1.0              30   
10574350             2       1.0        1.0               2   
16535678             2       1.0        1.0               2   
19123620             7       3.0        2.0               2   
...                ...       ...        ...             ...   
6973383              1       1.0        2.0               5   
18331809             6       2.0        2.0              30   
32385434             4       1.0        3.0              30   
8993307              4       0.0        1.0               2   
1141986              4       1.0        1.0             360   

          review_scores_rating  
room_id                         
13284332                  98.0  
38699317                   0.0  
10574350                  96.0  
16535678       

In [14]:
print(num_airbnb_df.predict(X_pred))

[219.26666667 185.16981132 136.46153846 ...  32.         175.
 175.        ]


Random Forest

In [15]:
#encode_X_train = X_train.copy()
#encode_X_pred = X_pred.copy()

rf_airbnb_model = RandomForestRegressor(n_estimators = 250, random_state = 42, max_depth = 20, criterion = 'mse', 
                                        min_samples_split = 2)
rf_airbnb_model.fit(X_train, Y_train)

RandomForestRegressor(max_depth=20, n_estimators=250, random_state=42)

In [16]:
predict_total = rf_airbnb_model.predict(X_pred)
mean_absolute_error(Y_pred, predict_total)

117.9431949547896

In [17]:
mean_squared_error(Y_pred, predict_total, squared=False)

446.14874889472827

In [18]:
mean_squared_error(Y_pred, predict_total)

199048.70614033128

In [19]:
print(f'R2: {rf_airbnb_model.score(X, Y)}')

R2: 0.29483273131234966


Correlation Matrix

In [ ]:
file_path = Path("/Users/Emily/documents/class_data/final_group_airbnb/Resources/listings.csv")
original_df = pd.read_csv(file_path, index_col=0)
original_df.head()

In [ ]:
# heatmap
cmx2 = original_df.corr()
cmx2 = abs(cmx2)
f, ax = plt.subplots(figsize=(20, 9))
sns.heatmap(cmx2, vmax=0.8, vmin=0.05, annot=True)

In [ ]:
# heatmap
cmx = airbnb_df.corr()
cmx = abs(cmx)
f, ax = plt.subplots(figsize=(20, 9))
sns.heatmap(cmx, vmax=0.8, vmin=0.05, annot=True)